# Algo Trading News or Noise

## Imports
---

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pathlib import Path
%matplotlib inline
import matplotlib.pyplot as plt
import plotly.express as px
import hvplot.pandas
from dotenv import load_dotenv
import panel as pn
from panel.interact import interact
from panel import widgets
import alpaca_trade_api as tradeapi
import quandl
import os
import requests
import ccxt
import time

## Read in data
---

In [51]:
ETF_csv = Path("Resources/ETFs0.csv")
ETF_df = pd.read_csv(ETF_csv, index_col='Unnamed: 0', infer_datetime_format=True, parse_dates=True)
ETF_df = ETF_df.drop(ETF_df.index[0])
ETF_df.index = pd.to_datetime(ETF_df.index, utc=True).date
ETF_df.sort_index(inplace=True)
ETF_df.tail()

,SPY,XLF,XLK,XLU,XLI,XLV,XLY,XLP,XLB,XOP,XHB,XME,XRT,ETH,LTC
2020-12-24,368.9899,28.94,129.05,61.19,88.01,111.48,157.91,66.44,71.58,58.99,59.40,32.57,64.740,20.09,39.30
2020-12-28,372.1400,29.07,130.52,61.48,88.09,111.72,159.68,66.97,71.28,57.92,58.41,32.40,65.070,20.25,39.41
2020-12-29,371.4800,28.97,129.89,61.47,87.52,112.24,159.74,67.42,71.12,57.35,57.92,32.26,64.275,20.02,39.03
2020-12-30,371.9300,29.11,129.83,61.79,88.13,112.24,160.69,67.13,72.10,59.32,58.16,33.40,64.910,20.26,38.68
2020-12-31,373.8500,29.47,130.07,62.70,88.59,113.43,160.77,67.05,72.38,58.49,57.63,33.43,64.330,20.21,38.91


## Set analysis inputs
---


In [53]:
ticker = input("Please enter the symbol name in caps:\n")
# Set intial capital
initial_capital = input("Please input the amount of capital you have (USD):\n")
initial_capital = int(initial_capital)
# 
share_size = input("Please input the share size you wish to trade:\n")
share_size = int(share_size)
# SMA windows
# EMA windows
# BB window
# RSI window
# MACD windows
# initial capital (for backtesting)

Please enter the symbol name in caps:
 SPY
Please input the amount of capital you have (USD):
 10000
Please input the share size you wish to trade:
 500



## Signal generation
---
### SMA
*Simple Moving Average for a given lookback window*

In [54]:
# Grab `ticker` from the dataset
SMA_df = ETF_df[f'{ticker}'].copy().to_frame()

In [56]:
# Set the short window and long windows
short_window = input("Please choose a short window option: 20, 50 days\n")
short_window = int(short_window)
long_window = input("Please choose a long window option: 100, 200 days\n")
long_window = int(long_window)

# Generate the short and long moving averages (50 and 100 days, respectively)
SMA_df["SMA_short"] = SMA_df[{ticker}].rolling(window=short_window).mean()
SMA_df["SMA_long"] = SMA_df[{ticker}].rolling(window=long_window).mean()

Please choose a short window option: 20, 50 days
 20
Please choose a long window option: 100, 200 days
 100


In [57]:
# Initialize the new `Signal` column
SMA_df["signal"] = 0.0

# Generate the trading signal -1, 1,
# where -1 is when the SMA50 is under the SMA100, and
# where 1 is when the SMA50 is higher (or crosses over) the SMA100
SMA_df["signal"][short_window:] = np.where(
    SMA_df["SMA_short"][short_window:] < SMA_df["SMA_long"][short_window:], -1.0, 1.0
)

In [58]:
# Calculate the points in time at which a position should be taken, 1 or -1
SMA_df["entry/exit"] = SMA_df["signal"].diff()

In [59]:
# Tidy up
SMA_df.dropna(inplace=True)
SMA_df.tail()

,SPY,SMA_short,SMA_long,signal,entry/exit
2020-12-24,368.9899,367.568975,347.260294,1.0,0.0
2020-12-28,372.1400,367.993475,347.661094,1.0,0.0
2020-12-29,371.4800,368.467475,348.032794,1.0,0.0
2020-12-30,371.9300,368.765475,348.406594,1.0,0.0
2020-12-31,373.8500,369.119980,348.789594,1.0,0.0


---
### EMA
*Exponential-weighted Moving Average for a given lookback window*

In [ ]:
# Grab `ticker` from the dataset
EMA_df = ETF_df[f'{ticker}'].copy().to_frame()

In [ ]:
# Set short and long windows
short_window = input("Please choose a short window option: 20, 50 days\n")
short_window = int(short_window)
long_window = input("Please choose a long window option: 100, 200 days\n")
long_window = int(long_window)
# Construct a `Fast` and `Slow` Exponential Moving Average from short and long windows, respectively
EMA_df['EMA_short'] = EMA_df[{ticker}].ewm(halflife=short_window).mean()
EMA_df['EMA_long'] = EMA_df[{ticker}].ewm(halflife=long_window).mean()

In [11]:
# Initialize signal column
EMA_df["signal"] = 0.0

# Construct a crossover trading signal
# where -1 is when the EMA50 is under the EMA200, and
# where 1 is when the EMA50 is higher (or crosses over) the EMA200
EMA_df["signal"][short_window:] = np.where(
    EMA_df["EMA_short"][short_window:] < EMA_df["EMA_long"][short_window:], -1.0, 1.0
)

In [12]:
# Calculate the points in time at which a position should be taken, 1 or -1
EMA_df["entry/exit"] = EMA_df["signal"].diff()

In [13]:
# Tidy up
EMA_df.dropna(inplace=True)
EMA_df.tail()

,SPY,EMA_short,EMA_long,signal,entry/exit
2020-12-24,368.9899,356.165678,324.284557,1.0,0.0
2020-12-28,372.1400,356.709822,324.615119,1.0,0.0
2020-12-29,371.4800,357.212948,324.938838,1.0,0.0
2020-12-30,371.9300,357.714265,325.263430,1.0,0.0
2020-12-31,373.8500,358.263908,325.599042,1.0,0.0


---
### BBD
*Bollinger Bands for a given lookback window*

In [14]:
# Grab `ticker` from the dataset
BBD_df = ETF_df[f'{ticker}'].copy().to_frame()

In [15]:
# Set bollinger band window
bollinger_window = input("Please choose a 10, 20, 50 bollinger lookback window\n")
bollinger_window = int(bollinger_window)

Please choose a 10, 20, 50 bollinger lookback window
 20


In [23]:
# Calculate rolling mean and standard deviation
BBD_df['BBD_mid'] = BBD_df[{ticker}].rolling(window=bollinger_window).mean()
BBD_df['BBD_std'] = BBD_df[{ticker}].rolling(window=bollinger_window).std()

# Calculate upper and lowers bands of bollinger band
BBD_df['BBD_upper']  = BBD_df['BBD_mid'] + (BBD_df['BBD_std'] * 1)
BBD_df['BBD_lower']  = BBD_df['BBD_mid'] - (BBD_df['BBD_std'] * 1)

In [24]:
# Construct a crossover trading signal
# where 1 is when the close is under the lower bound, and
# where -1 is when the close is higher the upper bound
BBD_df["long"] = np.where(
    BBD_df[ticker] < BBD_df["BBD_lower"], 1.0, 0.0)
BBD_df["short"] = np.where(
    BBD_df[ticker] > BBD_df["BBD_upper"], 1.0, 0.0)

In [25]:
# Calculate bollinger band trading signal
BBD_df['signal'] = BBD_df['long'] - BBD_df['short']
BBD_df["entry/exit"] = BBD_df["signal"].diff()

In [26]:
# Tidy up
BBD_df.dropna(inplace=True)
BBD_df.tail()

,SPY,BBD_mid,BBD_std,BBD_upper,BBD_lower,long,short,signal,entry/exit
2020-12-24,368.9899,367.568975,2.439630,370.008605,365.129345,0.0,0.0,0.0,0.0
2020-12-28,372.1400,367.993475,2.460381,370.453856,365.533094,0.0,1.0,-1.0,-1.0
2020-12-29,371.4800,368.467475,2.136852,370.604327,366.330623,0.0,1.0,-1.0,0.0
2020-12-30,371.9300,368.765475,2.185264,370.950739,366.580211,0.0,1.0,-1.0,0.0
2020-12-31,373.8500,369.119980,2.406666,371.526646,366.713314,0.0,1.0,-1.0,0.0


---
### RSI
*Relative Strength Index for a given lookback window*

In [27]:
########################### TO DO ########################### 

---
### MACD
*Moving Average Convergence/ Divergence for a given lookback window*

In [28]:
########################### TO DO ########################### 

## Backtesting
---
### Simulated portfolio
*Evaluate historic signal performance*

In [ ]:
# Take a 500 share position where the dual moving average crossover is 1 (SMA50 is greater than SMA100)
SMA_df["position"] = share_size * SMA_df["signal"]
EMA_df["position"] = share_size * EMA_df["signal"]
BBD_df["position"] = share_size * BBD_df["signal"]

In [ ]:
# Find the points in time where a 500 share position is bought or sold
SMA_df["entry/exit position"] = SMA_df["position"].diff()
EMA_df["entry/exit position"] = EMA_df["position"].diff()
BBD_df["entry/exit position"] = BBD_df["position"].diff()

In [ ]:
# Multiply share price by entry/exit positions and get the cumulatively sum
SMA_df["portfolio holdings"] = (SMA_df[f'{ticker}'] * SMA_df["entry/exit position"].cumsum())
EMA_df["portfolio holdings"] = (EMA_df[f'{ticker}'] * EMA_df["entry/exit position"].cumsum())
BBD_df["portfolio holdings"] = (BBD_df[f'{ticker}'] * BBD_df["entry/exit position"].cumsum())

In [ ]:
# Subtract the initial capital by the portfolio holdings to get the amount of liquid cash in the portfolio
SMA_df["portfolio cash"] = (initial_capital - (SMA_df[f'{ticker}'] * SMA_df["entry/exit position"]).cumsum())
EMA_df["portfolio cash"] = (initial_capital - (EMA_df[f'{ticker}'] * EMA_df["entry/exit position"]).cumsum())
BBD_df["portfolio cash"] = (initial_capital - (BBD_df[f'{ticker}'] * BBD_df["entry/exit position"]).cumsum())

In [ ]:
# Get the total portfolio value by adding the cash amount by the portfolio holdings (or investments)
SMA_df["portfolio total"] = (SMA_df["portfolio cash"] + SMA_df["portfolio holdings"])
EMA_df["portfolio total"] = (EMA_df["portfolio cash"] + EMA_df["portfolio holdings"])
BBD_df["portfolio total"] = (BBD_df["portfolio cash"] + BBD_df["portfolio holdings"])

In [ ]:
# Calculate the portfolio daily returns
SMA_df["portfolio daily returns"] = SMA_df["portfolio total"].pct_change()
EMA_df["portfolio daily returns"] = EMA_df["portfolio total"].pct_change()
BBD_df["portfolio daily returns"] = BBD_df["portfolio total"].pct_change()

In [ ]:
# Calculate the cumulative returns
SMA_df["portfolio cumulative returns"] = (1 + SMA_df["portfolio daily returns"]).cumprod() - 1
EMA_df["portfolio cumulative returns"] = (1 + EMA_df["portfolio daily returns"]).cumprod() - 1
BBD_df["portfolio cumulative returns"] = (1 + BBD_df["portfolio daily returns"]).cumprod() - 1

In [ ]:
# Tidy up
SMA_df.dropna(inplace=True)
EMA_df.dropna(inplace=True)
BBD_df.dropna(inplace=True)

---
### Descriptive statistics
*Evaluate backtesting*

In [ ]:
# Prepare DataFrame for metrics
metrics = [
    'Annual Return (%)',
    'Cumulative Returns (%)',
    'Annual Volatility (%)',
    'Sharpe Ratio',
    'Sortino Ratio',
    'Max Drawdown (%)']

columns = ['SMA', 'EMA', 'BBD', 'RSI', 'MACD']

In [ ]:
# Initialize the DataFrame with index set to evaluation metrics and columns
eval_df = pd.DataFrame(index=metrics, columns=columns)
eval_df.index.name = f'{ticker}' 

In [ ]:
# Calculate cumulative return
eval_df.loc['Cumulative Returns (%)', 'SMA'] = round(SMA_df['portfolio cumulative returns'][-1],2)
eval_df.loc['Cumulative Returns (%)', 'EMA'] = round(EMA_df['portfolio cumulative returns'][-1],2)
eval_df.loc['Cumulative Returns (%)', 'BBD'] = round(BBD_df['portfolio cumulative returns'][-1],2)

In [ ]:
# Calculate annualized return
eval_df.loc['Annual Return (%)', 'SMA'] = round((SMA_df['portfolio daily returns'].mean() * 252),2)
eval_df.loc['Annual Return (%)', 'EMA'] = round((EMA_df['portfolio daily returns'].mean() * 252),2)
eval_df.loc['Annual Return (%)', 'BBD'] = round((BBD_df['portfolio daily returns'].mean() * 252),2)

In [ ]:
# Calculate annual volatility
eval_df.loc['Annual Volatility (%)', 'SMA'] = round(SMA_df['portfolio daily returns'].std() * np.sqrt(252),2)
eval_df.loc['Annual Volatility (%)', 'EMA'] = round(EMA_df['portfolio daily returns'].std() * np.sqrt(252),2)
eval_df.loc['Annual Volatility (%)', 'BBD'] = round(BBD_df['portfolio daily returns'].std() * np.sqrt(252),2)

In [ ]:
# Calculate Sharpe Ratio
eval_df.loc['Sharpe Ratio', 'SMA'] = round((SMA_df['portfolio daily returns'].mean() * 252) / (SMA_df['portfolio daily returns'].std() * np.sqrt(252)), 2)
eval_df.loc['Sharpe Ratio', 'EMA'] = round((EMA_df['portfolio daily returns'].mean() * 252) / (EMA_df['portfolio daily returns'].std() * np.sqrt(252)), 2)
eval_df.loc['Sharpe Ratio', 'BBD'] = round((BBD_df['portfolio daily returns'].mean() * 252) / (BBD_df['portfolio daily returns'].std() * np.sqrt(252)), 2)

In [ ]:
# Calculate Sortino Ratio

In [44]:
eval_df

NameError: name 'eval_df' is not defined

## Deep learning
---
### All signals dataframe

In [42]:
# Combine signals
signals_df = pd.concat([ETF_df[f'{ticker}'], 
                        SMA_df['SMA_long'], SMA_df['SMA_short'],
                        EMA_df['EMA_long'], EMA_df['EMA_short'],
                        BBD_df['BBD_mid'], BBD_df['BBD_std'], BBD_df['BBD_upper'], BBD_df['BBD_lower']
                       ], axis=1)

In [43]:
signals_df.dropna(inplace=True)
signals_df.tail()

,SPY,SMA_long,SMA_short,EMA_long,EMA_short,BBD_mid,BBD_std,BBD_upper,BBD_lower
2020-12-24,368.9899,347.260294,367.568975,324.284557,356.165678,367.568975,2.439630,370.008605,365.129345
2020-12-28,372.1400,347.661094,367.993475,324.615119,356.709822,367.993475,2.460381,370.453856,365.533094
2020-12-29,371.4800,348.032794,368.467475,324.938838,357.212948,368.467475,2.136852,370.604327,366.330623
2020-12-30,371.9300,348.406594,368.765475,325.263430,357.714265,368.765475,2.185264,370.950739,366.580211
2020-12-31,373.8500,348.789594,369.119980,325.599042,358.263908,369.119980,2.406666,371.526646,366.713314
